## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsa Matruh,EG,2021-04-25 18:37:22,31.3525,27.2453,64.4,82,40,14.97,scattered clouds
1,1,Saint-Philippe,RE,2021-04-25 18:35:44,-21.3585,55.7679,78.8,73,20,19.57,light rain
2,2,Bocaiuva,BR,2021-04-25 18:37:23,-17.1078,-43.8150,87.8,30,0,5.75,clear sky
3,3,Hermanus,ZA,2021-04-25 18:30:39,-34.4187,19.2345,61.0,87,100,0.67,overcast clouds
4,4,Amberley,US,2021-04-25 18:37:24,39.2048,-84.4280,63.0,48,40,6.91,scattered clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Philippe,RE,2021-04-25 18:35:44,-21.3585,55.7679,78.80,73,20,19.57,light rain
2,2,Bocaiuva,BR,2021-04-25 18:37:23,-17.1078,-43.8150,87.80,30,0,5.75,clear sky
8,8,Rikitea,PF,2021-04-25 18:37:25,-23.1203,-134.9692,77.70,75,100,7.94,light rain
11,11,Atuona,PF,2021-04-25 18:37:26,-9.8000,-139.0333,80.04,71,53,16.04,broken clouds
15,15,Avarua,CK,2021-04-25 18:34:16,-21.2078,-159.7750,80.60,78,40,9.22,moderate rain
16,16,Kapaa,US,2021-04-25 18:37:27,22.0752,-159.3190,75.99,73,40,13.80,scattered clouds
18,18,Kimbe,PG,2021-04-25 18:32:48,-5.5502,150.1429,75.38,90,92,2.73,light rain
19,19,Buala,SB,2021-04-25 18:37:28,-8.1450,159.5921,79.57,84,29,2.46,scattered clouds
20,20,Puerto Ayora,EC,2021-04-25 18:33:21,-0.7393,-90.3518,82.40,65,20,19.57,few clouds
31,31,Bengkulu,ID,2021-04-25 18:37:32,-3.8004,102.2655,75.99,89,41,4.61,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Date                   207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna().copy()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
2,Bocaiuva,BR,87.80,clear sky,-17.1078,-43.8150,
8,Rikitea,PF,77.70,light rain,-23.1203,-134.9692,
11,Atuona,PF,80.04,broken clouds,-9.8000,-139.0333,
15,Avarua,CK,80.60,moderate rain,-21.2078,-159.7750,
16,Kapaa,US,75.99,scattered clouds,22.0752,-159.3190,
18,Kimbe,PG,75.38,light rain,-5.5502,150.1429,
19,Buala,SB,79.57,scattered clouds,-8.1450,159.5921,
20,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,
31,Bengkulu,ID,75.99,scattered clouds,-3.8004,102.2655,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna().copy()

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure()
# center=(30.0, 31.0), zoom_level=1.5
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))